In [6]:
import onnxruntime as ort
import numpy as np
import scipy.special
from PIL import Image


# 预处理图像
def preprocess_image(image, resize_size=256, crop_size=224, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
  image = image.resize((resize_size, resize_size), Image.BILINEAR)
  w, h = image.size
  left = (w - crop_size) / 2
  top = (h - crop_size) / 2
  image = image.crop((left, top, left + crop_size, top + crop_size))
  image = np.array(image).astype(np.float32)
  image = image / 255.0
  image = (image - mean) / std
  image = np.transpose(image, (2, 0, 1))
  image = image.reshape((1,) + image.shape)
  return image


# 模型加载 2分
session = ort.InferenceSession('resnet.onnx')


# 加载类别标签
labels_path = 'labels.txt'
with open(labels_path) as f:
  labels = [line.strip() for line in f.readlines()]


# 获取模型输入和输出的名称
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name


# 加载图片 2分
image = Image.open('img_test.jpg').convert('RGB')


# 预处理图片 2分
processed_image = preprocess_image(image)


# 确保输入数据是 float32 类型
processed_image = processed_image.astype(np.float32)


# 进行图片识别 2分
output = session.run([output_name], {input_name: processed_image})[0]

display(output)

# 应用 softmax 函数获取概率 2分
probabilities = scipy.special.softmax(output, axis=-1)


# 获取最高的5个概率和对应的类别索引 3分
top5_idx = np.argsort(probabilities[0])[-5:][::-1]

display(top5_idx)

top5_prob = probabilities[0][top5_idx]

# 打印结果
print("Top 5 predicted classes:")
for i in range(5):
  print(f"{i+1}: {labels[top5_idx[i]]} - Probability: {top5_prob[i]}")


array([[ 3.16601586e+00, -1.93939292e+00, -8.13415766e-01,
         8.01804900e-01,  1.29306161e+00,  4.04156351e+00,
         3.01537657e+00, -1.91776502e+00, -1.78754950e+00,
         2.42030716e+00, -2.67878342e+00, -1.90435278e+00,
        -2.11378765e+00, -1.00755095e+00, -2.25768185e+00,
        -1.76190209e+00, -1.76274836e+00, -2.66670442e+00,
        -2.53027868e+00, -4.57103443e+00, -7.61568844e-01,
         1.81299770e+00, -4.54728842e-01,  2.23174667e+00,
        -1.32510990e-01, -9.14915621e-01,  1.20954013e+00,
        -1.85026753e+00, -1.05334544e+00,  8.34507167e-01,
         6.13426030e-01, -7.39417493e-01,  9.80048716e-01,
         1.57937729e+00,  9.25156116e-01,  7.13568747e-01,
         3.09202743e+00, -7.80150890e-02, -7.58827150e-01,
         3.26579958e-01, -6.83769822e-01, -1.52953124e+00,
         1.98197350e-01,  1.84872878e+00,  4.79753166e-01,
         1.68322623e-02, -1.10750711e+00,  3.71496177e+00,
         4.83494473e+00,  1.03355229e+00,  1.25584412e+0

array([101, 385, 386, 366, 346], dtype=int64)

Top 5 predicted classes:
1: tusker - Probability: 0.931233286857605
2: Indian elephant - Probability: 0.06785939633846283
3: African elephant - Probability: 0.0009034110116772354
4: gorilla - Probability: 4.78091294553451e-07
5: water buffalo - Probability: 3.9817274455344887e-07
